In [6]:
import json
import os
import shutil
import subprocess
import tempfile
import glob
import pandas as pd
from functools import reduce
import re
import pandas as pd
import time

# reduce new function https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

commonpeaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

metadata = '/home/joaquin/projects/methylation/data/commonData/ids_data_allReplicates_methylation.json'
basePathDataFolder = '/home/joaquin/projects/methylation/data'
narrowPeakLocationFolders = ['tfs_rep_1','tfs_rep_3_input_from_rep_2', 'tfs_rep_2', 'tfs_rep_4']
specificPathsSumary = {}
with open(metadata) as jsonMetadata:
     experimentsClasification = json.load(jsonMetadata)['experiments']
for experiment in experimentsClasification:
    specificPathsSumary[experiment['condition']] = {}
    for metState in ['direct', 'amplified']:
        specificPathsSumary[experiment['condition']][metState] = {}
        for exptype in ['sample', 'input']:
            specificPathsSumary[experiment['condition']][metState][exptype] = {}
            for replicate, number in zip(
                experiment[metState],
                ['replicate1','replicate2', 'replicate3']
            ):
            
                expeId, expPath = replicate[number][0][exptype].strip().split(',')
        # if it is a missing experiment dont continue with the analisys
                if 'MISSING' in expeId:
                    print(number, expPath, expeId)
                    continue
                for possiblenarrowPeakFolder in narrowPeakLocationFolders:
                    narrowPeakFolder = None
                    path = os.path.join(basePathDataFolder,possiblenarrowPeakFolder,expPath)
        # make a list of the files in each posible directory. Try and continue if the experiment was not 
        # done for the replicate. 
                    try:
                        filesInFolder = os.listdir(path)
                    except FileNotFoundError:
                        continue
        # search for the specific experiment id inside the folder in the names to check if it is the correct folder
        # and stop searching if it is inside

                    for file in filesInFolder:
                        if 'html' in file:
                            fileid=file
    #                 print(fileid)
    #                 print(expeId)
                    if fileid.startswith(expeId):
                        narrowPeakFolder = possiblenarrowPeakFolder
                        break

                narrowpeakFileOriginalPath = os.path.join(
                    basePathDataFolder,narrowPeakFolder,expPath
                )
    #             specificPathsSumary[experiment['condition']][metState].append((narrowPeakFolder,narrowpeakFileOriginalPath))
                specificPathsSumary[experiment['condition']][metState][exptype][number] = narrowpeakFileOriginalPath 
        

replicate3 MYC3/amplified/6/Mock MISSING
replicate2 MYCH7/direct/24/Mock MISSING


In [10]:
def calculationGemSummary(folder):
    with cd(folder):

        significant = None

        try:
            with open('GEM_Log.txt', 'r') as logGem:
                for line in logGem:
                    # if '_IP' in line:
                    #    print(line)
                    # if '_CTRL' in line:
                    #    print(line)
                    if 'Significant:' in line:
                        print(line)
                        significant = int(line.split('\t')[1])
                    # if 'Insignificant:' in line:
                    #    ins = line
                    # if 'Filtered:' in line:
                    #    fil = line
            if significant == None:
                return 'No significant Peaks'
            return significant
        except:
            print('no gem done')

def calculationBowtieSummary(filepath):
    filename = os.path.join(filepath,'bowtie2stats.txt')
    with open(filename, 'r') as bowstats:
        for line in bowstats:
            reads = re.search(r'([\d]+) reads; of these:',line)
            regular = re.search(r'([\d,\.]+)% overall alignment rate',line)

        return reads.group(1),regular.group(1)

In [13]:
import os
class cd:
    """Context manager for changing the current working directory"""

    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)


In [18]:
nmrpeak = 'peak_aling_mets.tsv'

with open(nmrpeak, 'w') as file:
    file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                        'experiment','metState','exptype','replicate','workingFolder','reads','alingpercent','Significant'
                    )
                         )
    for experiment in specificPathsSumary:
        for metState in specificPathsSumary[experiment]:
            for exptype in specificPathsSumary[experiment][metState]:
                for replicate in specificPathsSumary[experiment][metState][exptype]:
                    workingFolder = specificPathsSumary[experiment][metState][exptype][replicate]

                    reads, alingpercent = calculationBowtieSummary(workingFolder)
                    Significant = calculationGemSummary(workingFolder)

                    file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                        experiment,metState,exptype,replicate,workingFolder,reads,alingpercent,Significant
                    )
                         )

       

Significant:	23166

Significant:	23171

Significant:	27466

Significant:	19752

Significant:	15300

Significant:	17205

Significant:	12641

Significant:	34339

Significant:	44675

no gem done
no gem done
no gem done
Significant:	10130

Significant:	10250

Significant:	12880

Significant:	13867

Significant:	8686

Significant:	9744

Significant:	23679

Significant:	35074

Significant:	38525

no gem done
no gem done
no gem done
Significant:	22235

Significant:	21324

Significant:	25294

Significant:	9622

Significant:	9424

Significant:	12318

Significant:	17590

Significant:	35071

Significant:	42193

no gem done
no gem done
no gem done
Significant:	20071

Significant:	19799

Significant:	22501

Significant:	14437

Significant:	8341

Significant:	9953

Significant:	33412

Significant:	33717

Significant:	34386

no gem done
no gem done
no gem done
Significant:	19819

Significant:	20087

Significant:	22668

Significant:	13053

Significant:	10481

Significant:	12531

Significant:	41292

Si